<a href="https://colab.research.google.com/github/JakobSchauser/BachelorProject-IceCube-ML/blob/main/Second_working_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install spektral -q

     |████████████████████████████████| 122kB 5.4MB/s 


In [8]:
# Mainly following the semi-official example code from 
# https://github.com/danielegrattarola/spektral/blob/master/examples/node_prediction/citation_simple_gc.py
import numpy as np

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

from spektral.data.loaders import BatchLoader, SingleLoader, DisjointLoader
from spektral.datasets.citation import Citation
from spektral.layers import GCNConv, ECCConv, GlobalSumPool, GlobalMaxPool
from spektral.transforms import LayerPreprocess, AdjToSpTensor
from spektral.data import Dataset, Graph
from spektral.models import GeneralGNN



from spektral.data import Dataset
from tqdm.notebook import tqdm

import tensorflow as tf

#Lifehack
true = True
false = False

In [9]:

# Start out by loading data
class dat(Dataset):
  def __init__(self,n = 1,**kwargs):
    self.n = n
    super().__init__(**kwargs)
  def read(self):
    # path = "/content/drive/MyDrive/Bachelor Project - IceCube ML/generatedDataEnergy100000 0.npz"
    path = "/content/drive/MyDrive/Bachelor Project - IceCube ML/data.dat"
    dataset = np.load(path,allow_pickle = True)#["arr_0"]
    graphs = []
    for g in dataset:
      graphs.append(g)
    return np.array(dataset)

In [10]:
dataset = dat()
print(dataset,"\n",dataset[0])

dat(n_graphs=100000) 
 Graph(n_nodes=34, n_node_features=5, n_edge_features=None, n_labels=7)


In [11]:
# Setting up the model training parameter

learning_rate = 0.05    
epochs = 1000
early_stopping_patience = 50

batch_size = 30

N_nodes = dataset.n_nodes
N_features = dataset.n_node_features

n_out = dataset.n_labels

In [31]:
# Trying to recreate a DCN from 
# https://www.asimovinstitute.org/neural-network-zoo/

# Define the model
size = 32

X_in = Input(shape = (N_features,), name = "X_in")
A_in = Input(shape = (None,), sparse = True, name = "A_in")
I_in = Input(shape = (), name="segment_ids_in", dtype=tf.int32)

x = GCNConv(size,activation="relu")([X_in,A_in])
x = GCNConv(size//2,activation="relu")([x,A_in])
x = GCNConv(size//2,activation="relu")([x,A_in])
x = GCNConv(size//2,activation="relu")([x,A_in])
x = GCNConv(batch_size,activation="relu")([x,A_in])

# x = Dense(size,activation="relu")(x)
# x = Dense(size,activation="relu")(x)
# x = Dense(size)(x)

x_out = GlobalSumPool()([x,I_in])

output = Dense(n_out)(x_out)
# Build the model
model = Model(inputs = [X_in,A_in,I_in], outputs = output)


acc_func = tf.keras.metrics.mean_absolute_error
loss_func = tf.losses.mean_squared_error


# Ignore this for the time being
# class model(Model):
#   def __init__(self,n_out):
#     super().__init__(activation=activation, aggregate = "mean")
#     self.n_out = n_out
  
#   def build(self, input_shape):
#     n_in = input_shape[0]
#     self.weights = self.add_weight(shape=(n_in,self.n_out))

#   def call(self, inputs):
#     x, a = inputs

#     x = tf.matmul(x, self.weights)

#   def update(self,embeddings):
#     return self.activation(embeddings)
# output = GCNConv(n_out, activation='softmax',use_bias = False)([x_in, a_in]) # maybe add use_bias = False

# model = Model(inputs=[x_in, a_in], outputs=output)
# model = GeneralGCN(dataset.n_labels,activation="softmax",) # Using softmax now. Maybe gelu would be better. Maybe add pool = sum

optimizer = Adam(lr=learning_rate)


split = int(len(dataset)*0.8)
traindat, testdat = dataset[:split], dataset[split:]


train_loader = DisjointLoader(traindat,epochs=epochs,batch_size = batch_size,shuffle = True) 
test_loader  = DisjointLoader(testdat,epochs=1,batch_size = batch_size,shuffle = True)
# model.summary()


# Training step function
@tf.function(input_signature=train_loader.tf_signature(),experimental_relax_shapes=True)
def train_on_batch(inputs,target): 
  with tf.GradientTape() as tape:
    predictions = model(inputs,training = True)
    loss = loss_func(target,predictions) + sum(model.losses)#+0.01
    acc = acc_func(target,predictions)
  
  gradients = tape.gradient(loss,model.trainable_variables)
  optimizer.apply_gradients(zip(gradients,model.trainable_variables))
  return loss, acc

# Evaluating function 
def evaluate(loader):
  step = 0
  results = []
  for batch in loader:
    step += 1
    inputs, target = batch
    predictions = model(inputs, training = False)
    loss = loss_func(target,predictions) + sum(model.losses)+0.01
    acc = acc_func(target,predictions)
    l = [np.mean(loss),np.mean(acc)]
    results.append(l)
    # if step == loader.steps_per_epoch:
    results = np.array(results)
    loss, acc = np.mean(results[:,0]), np.mean(results[:,1])
    return np.array([loss, acc])
  


#Doing the full training

def train_model(train_loader, test_loader):
  print('Training model.')

  epoch, step = 0,0
  results = []

  for batch in tqdm(train_loader,total = train_loader.steps_per_epoch):
    step += 1
    inputs, target = batch
    x,a,i = inputs
    # print(x)
    # target = target.reshape(-1, 1)
    loss, acc = train_on_batch(inputs,target)
    results.append((np.mean(loss),np.mean(acc)))

    if step == train_loader.steps_per_epoch:
      step = 0
      epoch += 1
      test_results = evaluate(test_loader)
      print(test_results)
      tarr = np.array(results)
      ploss = np.mean(tarr[:,0])
      pacc = np.mean(tarr[:,1])
      ptloss = test_results[0]
      ptacc = test_results[1]

      # print(ptloss,ptacc)
      print("Epoch: {} - ".format(epoch) + \
      "Train loss: {:.3} Train Acc: {:.3} - ".format(ploss,pacc) + \
      "Test loss: {:.3} Test Acc: {:.3}".format(ptloss,ptacc))

      if(epoch%2 == 1):
        print("A single guess is right now:")
        i,t = batch
        print(model(i,training = False),"for a correct", t)


In [32]:
# train_loader.steps_per_epoch

model.compile()
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
X_in (InputLayer)               [(None, 5)]          0                                            
__________________________________________________________________________________________________
A_in (InputLayer)               [(None, None)]       0                                            
__________________________________________________________________________________________________
gcn_conv_40 (GCNConv)           (None, 32)           160         X_in[0][0]                       
                                                                 A_in[0][0]                       
__________________________________________________________________________________________________
gcn_conv_41 (GCNConv)           (None, 16)           512         gcn_conv_40[0][0]         

In [33]:
train_model(train_loader,test_loader)

print('Evaluating model.')
eval_results = evaluate(test_loader)

print('Done.\n' 
      'Test loss: {}\n'
      'Test accuracy: {}'.format(*eval_results))


Training model.




[23709.68       72.69914]
Epoch: 1 - Train loss: 1.3e+14 Train Acc: 3.13e+05 - Test loss: 2.37e+04 Test Acc: 72.7
A single guess is right now:
tf.Tensor(
[[ 0.12440668  0.20931964  0.11185937 -0.39665404 -0.05907349 -0.43635502
   0.12454455]
 [ 0.12440668  0.20931964  0.11185937 -0.39665404 -0.05907349 -0.43635502
   0.12454455]
 [ 0.12440668  0.20931964  0.11185937 -0.39665404 -0.05907349 -0.43635502
   0.12454455]
 [ 0.12440668  0.20931964  0.11185937 -0.39665404 -0.05907349 -0.43635502
   0.12454455]
 [ 0.12440668  0.20931964  0.11185937 -0.39665404 -0.05907349 -0.43635502
   0.12454455]
 [ 0.12440668  0.20931964  0.11185937 -0.39665404 -0.05907349 -0.43635502
   0.12454455]
 [ 0.12440668  0.20931964  0.11185937 -0.39665404 -0.05907349 -0.43635502
   0.12454455]
 [ 0.12440668  0.20931964  0.11185937 -0.39665404 -0.05907349 -0.43635502
   0.12454455]
 [ 0.12440668  0.20931964  0.11185937 -0.39665404 -0.05907349 -0.43635502
   0.12454455]
 [ 0.12440668  0.20931964  0.11185937 -0.39

KeyboardInterrupt: ignored